In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree    # Decision Tree
from sklearn.linear_model import LogisticRegression # LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC # Support Vector Machine 
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import sklearn.feature_extraction.text as sk_text


## File Prep ##

In [ ]:
%%time
column_names=['Name', 'ScreenName', 'UserID', 'FollowersCount', 'FriendsCount', 'Location', 'Description', 'CreatedAt', 'StatusID', 'Language', 'Place', 'RetweetCount', 'FavoriteCount', 'Text']
tweets = pd.read_csv('clinton_trump_tweets.txt', sep="\t",  encoding="ISO-8859-1", header=None, names=column_names)
#print initial data
tweets = tweets.drop(['ScreenName', 'FollowersCount', 'FriendsCount', 'CreatedAt', 'StatusID','Language', 'Place', 'RetweetCount', 'FavoriteCount'], axis=1)
tweets.Location = tweets.Location.astype(str)
tweets.Text = tweets.Text.astype(str)

ground_truth = pd.read_table('clinton_trump_user_classes.txt', encoding ="ISO-8859-1", dtype=int, names=["UserID", "TrumpOrClinton"])



### Task 1.1 (10 pts): Remove all retweets first. Remove all users that have less than 20 tweets. You may want to keep the entire tweet content, including hashtags/handles. ###

In [ ]:
tweets = tweets[~tweets.Text.str.startswith('RT')]

In [ ]:
%%time
def keepHashMentions(text):
    hashMentions = []
    for word in text:
        hashMentions.extend(word)
    return " ".join(hashMentions).strip()
## keeping tweet including hashtags and mentions here ## 
tweets['HashMentions'] = tweets.Text.str.findall('(@\w+)|(#\w+)').apply(keepHashMentions)


### Remove all users that have less than 20 tweets. ###

In [ ]:
#1.2 keep tweets where UID appears 20+ times
tweets = tweets.groupby("UserID").filter(lambda x: len(x) >= 20)

In [ ]:
%%timeit
# Generate a list of mention/hashes that have a frequency of 20+ #
top_hash = pd.Series(tweets['HashMentions'].str.cat(sep=' ').split()).value_counts()
top20 = top_hash[top_hash>=20]
top20List = top20.index.tolist()

# Convert list to set so it will have quick lookup
top20Set = set(top20List)


# generate list of mention/hashes that occur 20+ times from our orginal list 
def removeUnder20Mentions(hashMentions):
    mentions =  hashMentions.split()
    mentionsOver20 = []
    for mention in mentions:
        if(mention in top20Set):
            mentionsOver20.append(mention)
    return " ".join(mentionsOver20)
tweets['HashMentions'] = tweets.HashMentions.apply(removeUnder20Mentions)

## Create new df merging hashmentions and UserID  ##

In [ ]:
hashMentionsMerged = tweets.groupby('UserID')['HashMentions'].apply(' '.join).reset_index()
tweetsWithNoHashMentions = tweets.drop('HashMentions', axis=1)
tweets = pd.merge(tweetsWithNoHashMentions, hashMentionsMerged, on='UserID')
tweets = tweets.drop_duplicates('UserID')


### #remove users that have no hashtag/handles ###

In [164]:
tweets = tweets[tweets.HashMentions.apply(lambda x:  bool(x and x.strip()))]
print(tweets.shape)

(6174, 7)


In [167]:
# Remove locations that are null 
tweets['LocationCleaned'] = tweets['Location'].apply(lambda x: x.split(',')[0])

tweetsLocations = tweets[tweets['Location'].notnull() & tweets['Description'].notnull() & tweets['HashMentions'].notnull()]

tweetsLocations = tweetsLocations[(tweetsLocations['LocationCleaned'] != 'nan') & (tweetsLocations['LocationCleaned'] != ' ') & (tweetsLocations['LocationCleaned'] != '')]
print(tweets.shape)

tweets = tweetsLocations[tweetsLocations['LocationCleaned'].isin(tweetsLocations['LocationCleaned'].value_counts().nlargest(300).index.tolist())]
tweets['LocationCleaned'] = tweetsLocations['LocationCleaned'].map({'NYC':'New York City','New York':'New York City', 'NY':'New York City', 'SF':'San Francisco', 'Hollywood':'Los Angeles'}).fillna(tweetsLocations['LocationCleaned'])

tweets['LocationCleaned'].value_counts()

(6174, 7)


New York City     534
Los Angeles       343
Washington        279
Chicago           234
London            218
United States     203
USA               171
San Francisco     139
Boston            128
Florida           125
California        119
Texas             119
Seattle           119
Brooklyn          116
Toronto           113
Atlanta           104
Houston           100
Austin             95
Portland           88
Philadelphia       80
Dallas             71
Denver             68
Las Vegas          68
Michigan           62
New Jersey         62
Ohio               61
San Diego          60
Cleveland          57
Canada             55
Columbus           54
                 ... 
Detroit            28
Missouri           28
Louisville         28
Milwaukee          27
Indiana            27
Oakland            27
Buffalo            27
Dublin             27
Arlington          26
Alabama            26
Connecticut        26
Manchester         25
Worldwide          25
Oklahoma           25
Global    

In [ ]:
aggregateTweetsHashtags = tweets.groupby('UserID')['HashMentions'].apply(lambda x: x.str.cat(sep=' '))
tweetsPrepareSKText = pd.DataFrame({'User_id': aggregateTweetsHashtags.index, 'All_hashtags': aggregateTweetsHashtags.values})
vectorizer = sk_text.TfidfVectorizer(max_features = 1000,
                             #min_df=100, 
                             #max_df=.8,
                             stop_words = ['UserID']
                             )
matrix = vectorizer.fit_transform(tweetsPrepareSKText.All_hashtags.values)
tdidf = matrix.toarray()
df_text = pd.DataFrame(matrix.todense(), index=aggregateTweetsHashtags.index, columns=vectorizer.get_feature_names())
df_text.head()

### Vectorize Description ###

In [168]:
vectorizerDescription = sk_text.TfidfVectorizer(max_features = 1000,
                             #min_df=100, 
                             #max_df=.8,
                             stop_words = ['UserID']
                            )

matrixDescription = vectorizerDescription.fit_transform(tweets.Description.values)
tdidfDescription = matrixDescription.toarray()
df_text_Description = pd.DataFrame(matrixDescription.todense(), index=tweets.UserID, columns=vectorizerDescription.get_feature_names())
df_text_Description.head()

,10,100,11,12,14,15,16,17,18,19,...,yes,yet,yoga,york,you,young,your,youtube,youtuber,yrs
UserID,,,,,,,,,,,,,,,,,,,,,
106568768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2447279666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
231921777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201499452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17547533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Vectorize Description ###

In [169]:
vectorizerLocation = sk_text.TfidfVectorizer(#max_features = 1000,
                             #min_df=100, 
                             #max_df=.8,
                             stop_words = ['UserID']                
                            )
matrixLocation = vectorizerLocation.fit_transform(tweets.LocationCleaned.values)
tdidfLocation = matrixDescription.toarray()
df_text_Location = pd.DataFrame(matrixLocation.todense(), index=tweets.UserID, columns=vectorizerLocation.get_feature_names())
df_text_Location.head()



,alabama,angeles,antonio,area,arizona,arlington,atlanta,austin,australia,baltimore,...,toronto,uk,united,usa,vancouver,vegas,virginia,washington,worldwide,york
UserID,,,,,,,,,,,,,,,,,,,,,
106568768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2447279666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
231921777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
201499452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.590197
17547533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


## Apply ground truths column to UserID of tweets Dataframe ##

In [170]:
#mergeGroundTruths = pd.merge(tweets, ground_truth, on = 'UserID')
#ground truth
dataFrameWithHashTagHandlesAndTruths = pd.merge(df_text,ground_truth, on = 'UserID')


##merge Description and Location into out dataframe 
dataFrameWithHashTagHandlesAndTruths = pd.merge(dataFrameWithHashTagHandlesAndTruths,df_text_Description,on = 'UserID')

dataFrameWithHashTagHandlesAndTruths = pd.merge(dataFrameWithHashTagHandlesAndTruths,df_text_Location,on = 'UserID')

## Implementing Logistic Regression on Location column, rest of models are below ##

### 1.1: Use train_test_split() to split data into training and test sets, where 20 percent of the records go to test set. ###

In [171]:
X = dataFrameWithHashTagHandlesAndTruths.drop(['UserID','TrumpOrClinton'], axis=1).head(4000)
# grabbing one of the location arrays: USA
y = dataFrameWithHashTagHandlesAndTruths.head(4000).TrumpOrClinton.values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# Task 1.2 (20 pts): Train Decision Tree, SVM, Logistic Regression, and Neural Networks #

## Decision Tree ##

In [ ]:
## Decision tree boilerplate
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred))

## SVM ##

In [173]:
clf = SVC(C=1.0, gamma='auto') 
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred))

0.47093906250000006
0.68625
0.5585637509266124
[[  0 251]
 [  0 549]]


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## LogisticRegression ##

In [174]:
logreg = LogisticRegression(solver='lbfgs') 
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)


print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred))

0.7790266625839231
0.77875
0.7551953343731428
[[103 148]
 [ 29 520]]


## Nueral Networks ##

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

scaler = StandardScaler()
scaler.fit(X_train)
X_train= scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Multilayer Perception Model: 30 30 30 is number of nerurons etc.
### Note: each feature does not have to have same number of neurons

In [ ]:
### hidden_layer_sizes: 3 hidden layers,  each has 30 neurons
### solver='adam' is a variation of gradient descent 
### max_iter determines the number of epochs

mlp = MLPClassifier(hidden_layer_sizes=(1000,1000,1000), solver='adam', max_iter=1000)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred))
print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred, labels=[0, 1]))


# Task 1.3 (20 pts): Train k-NN model. In your report describe the features that you used for k-NN. Perform parameter tuning on k-NN model. Apply 5-fold cross validation and use grid search to find the best K value for k-NN model. Set scoring metric to F1 score (F-measure). Use the best K value identified from grid search to train your k-NN model. Plot the F1 score against K value based on the results you achieved from grid search. #

In [ ]:
%%time
# X_train = X_train[:2220]
# y_train = y_train[:2220]
# X_test = X_test[:555]
# y_test = y_test[:555]


param_grid = dict(n_neighbors=range(1,21))
knn = KNeighborsClassifier(n_neighbors=1)
grid = GridSearchCV(knn, param_grid, cv=5, scoring='f1_weighted',n_jobs=2)
grid.fit(X_train, y_train)
# view the complete results
means = grid.cv_results_['mean_test_score']

In [ ]:
import matplotlib.pyplot as plt
# plot the results
plt.plot(range(1,21),means)
plt.xlabel('Value of K for KNN')
plt.ylabel('F1 score based on Cross-Validation')
plt.show()

In [ ]:
# identify the best model
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

In [ ]:
%%time
# train your model using all data and the best known parameters
#TODO: use best n_neighbors from grid search
knn = KNeighborsClassifier(n_neighbors=2, weights='uniform')
knn.fit(X_train, y_train)
y_pred =  knn.predict(X_test)

print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred))